##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 사용자 정의 layer

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/eager/custom_layers"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ko/tutorials/eager/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ko/tutorials/eager/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

신경망을 구축하기 위해서 고수준 API인 `tf.keras`를 사용하길 권합니다. 대부분의 텐서플로 API는 즉시 실행(eager execution)을 활성화할 수 있습니다.

In [ ]:
import tensorflow as tf

tf.enable_eager_execution()

## layer: 유용한 연산자 집합

머신러닝을 위한 코드를 작성하는 대부분의 시간동안 개별적인 연산과 변수를 조작하는 것보다는 고수준의 추상화 수준에서 작업하기를 원합니다.

많은 머신러닝 모델은 비교적 단순한 layer의 구성과 적층(stacking)으로 표현가능합니다. 또한 텐서플로는 여러 표준 layer 세트를 제공하므로 사용자 고유의 응용 프로그램에 관련된 layer를 처음부터 작성하거나, 기존 layer의 구성으로 쉽게 작성할 수 있습니다.

텐서플로는 [케라스](https://keras.io) API 의 풀패키지를 tf.keras package에 포함하고 있습니다. 케라스 layer는 모델을 구축하는데 매우 유용합니다.

In [ ]:
# tf.keras.layers 패키지에서 layer는 객체입니다. layer를 구성하려면 간단히 객체를 생성하십시오.
# 대부분의 layer는 첫번째 인수로 출력 차원(크기) 또는 채널을 취합니다.
layer = tf.keras.layers.Dense(100)
# 입력 차원의 수는 유추할 수 있기 때문에 종종 불필요합니다. 
# 일부 복잡한 모델에서는 수동으로 입력 차원의 수를 제공하는것이 유용할 수 있습니다.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

미리 구성되어있는 layer는 다음 [문서](https://www.tensorflow.org/api_docs/python/tf/keras/layers)에서 확인할 수 있습니다. Dense, Conv2D, LSTM, BatchNormalization, Dropout, 등을 포함하고 있습니다.

In [ ]:
# layer를 사용하기 위해서 간단하게 호출합니다.
layer(tf.zeros([10, 5]))

In [ ]:
# layer는 유용한 메서드를 내재하고있습니다. 예를 들어, `layer.variables`를 사용하여 layer안에 있는 모든 변수를 확인할 수 있으며, 
# `layer.trainable_variables`를 사용하여 학습가능한 변수를 확인할 수 있습니다. 
# 이번 케이스에서 완전 연결(fully-connected) 레이어는 가중치(weight)와 편향(biases)을 위한 변수를 가집니다. 
layer.variables

In [ ]:
# 또한 변수는 여러 접근자(accessors)를 통해 접근가능합니다. 
layer.kernel, layer.bias

## 사용자 정의 layer 구현
사용자 정의 layer를 구현하는 가장 좋은 방법은 tf.keras.Layer 클래스를 상속하고 다음과 같이 구현하는 것입니다.
  *  `__init__` , 모든 독립적인 입력값을 초기화를 할 수 있습니다.
  * `build`, 입력 텐서의 크기를 알고 나머지를 초기화 할 수 있습니다.
  * `call`, 정방향 연산(forward computation)을 진행 할 수 있습니다.

변수를 생성하기 위해 `build`가 호출되길 기다릴 필요가 없다는 것에 주목하세요. 또한 변수를 `__init__`에 생성할 수도있습니다. 그러나 `build`에 변수를 생성하는 유리한 점은 layer가 작동할 입력의 크기를 기준으로 나중에 변수를 만들 수 있습니다. 반면에, `__init__`에 변수를 생성하는것은 변수 생성에 필요한 크기가 명시적으로 지정되어야 함을 의미합니다.

In [ ]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs
    
  def build(self, input_shape):
    self.kernel = self.add_variable("kernel", 
                                    shape=[int(input_shape[-1]), 
                                           self.num_outputs])
    
  def call(self, input):
    return tf.matmul(input, self.kernel)
  
layer = MyDenseLayer(10)
print(layer(tf.zeros([10, 5])))
print(layer.trainable_variables)

변수를 생성하기 위해 `build`가 호출되길 기다릴 필요가 없다는 것에 주목하세요. 또한 변수를 `__init__`에 생성할 수도있습니다. 

다른 독자가 표준 layer의 동작을 잘 알고 있기 때문에, 가능한 경우 표준 layer를 사용하는것이 전체 코드를 읽고 유지하는데 더 쉽습니다. 만약 tf.keras.layers 또는 tf.contrib.layers에 없는 layer를 사용하기 원하면 [github issue](http://github.com/tensorflow/tensorflow/issues/new)에 이슈화하거나, 풀리퀘스트를 요청하세요.

## 모델: layer 구성

머신러닝 모델에서 대부분의 흥미로운 유사 layer(layer-likely)는 layer의 집합으로 구현되어집니다. 예를 들어, 레스넷(resnet)의 각 잔여 블록(residual block)은 합성곱(convolution), 배치 정규화(batch normalization), 쇼트컷(shortcut) 등으로 구성되어있습니다. 

layer 집합을 포함한 유사 layer를 생성하기위해 사용하는 메인 클래스는 tf.keras.Model입니다. 다음은 tf.keras.Model을 상속(inheritance)하여 구현합니다.

In [ ]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)

    
block = ResnetIdentityBlock(1, [1, 2, 3])
print(block(tf.zeros([1, 2, 3, 3])))
print([x.name for x in block.trainable_variables])

In [ ]:
 my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1)),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(2, 1, 
                                                      padding='same'),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(3, (1, 1)),
                               tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

# 다음 단계

이제 이전 노트북으로 돌아가서 선형 회귀 예제에 좀 더 나은 구조를 만들기 위해 layer와 모델을 적용할 수 있습니다.